## 합성곱 층을 부르는 단위
Convolution layer를 부르는 단위는 합성곱 신경망의 출처에 따라서 조금 다릅니다. 첫번째 표기 방식은 합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU)를 하나의 convolution layer로 보고, max pooling(nn.MaxPoold2d)은 pooling layer로 별도로 명명합니다.<br>
두번째 표기 방식은 합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)를 하나의 convolution layer로 명명합니다.<br>
이번 모델은 편의를 위해 두번째 표기 방식을 따르도록 하겠습니다.

In [1]:
import torch
import torch.nn as nn

1 x 1 x 28 x 28 사이즈의 임의의 tensor를 만듭니다.

In [2]:
# 배치 크기 x 채널 x 높이(height) x 너비(width) 크기의 tensor 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기: {}' .format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


첫번쨰 convolution layer를 구현해보겠습니다. 1 channel의 입력을 받아 32 channel을 뽑아내며, kernel size = 3이고 padding = 1입니다.

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


두번째 convolution layer는 32 channel의 입력을 64 channel로 출력합니다. kernel size = 3, padding = 1입니다.

In [4]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


이번에는 maxpooling을 구현해보겠습니다. 정수 하나를 인자로 넣으면 kernel size, stride가 둘 다 해당값으로 지정됩니다.

In [5]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 구현체를 연결하여 model 만들기
이제 앞서 선언한 구현체들을 연결하여 모델을 완성해보겠습니다. 우선 입력을 첫번째 convolution layer를 통과시키고, convolution layer를 통과시킨 후의 tensor의 크기를 보겠습니다.

In [6]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


output tensor가 1 batch size, 32 channel, 28 height, 28 width의 size를 갖는 결과를 확인할 수 있습니다. 32가 나온 이유는 out_channel을 32로 지정해주었기 때문입니다. 또한, 28 heigth, 28 width가 나온 이유는 3 x 3 kernel을 사용한 후 padding의 폭을 1로 설정하면 입력의 크기가 보존되기 때문입니다.<br>
이제 이를 maxpooling을 통과시키고난 후의 결과를 보겠습니다.

In [7]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


C:\Users\actgo\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


32 channel, 14 height, 14 width의 tensor가 되었습니다. 다시 두번째 convolution layer를 통과시킨 후의 결과를 보겠습니다.

In [8]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


이번에는 64 channel, 14 height, 14 width의 tensor가 결과로 나왔습니다. 64가 나온 이유는 conv2의 out_channel을 64로 지정해주었기 때문입니다. 또한 3 x 3 kernel을 이용해 convolution operation을 진행한 후 1의 폭만큼 padding을 진행하였기 때문에, height, width가 보존된 결과를 확인할 수 있었습니다. 이제 max pooling에 통과시킨 후의 결과를 보겠습니다.

In [9]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


이제 텐서를 펼치기에 앞서 n번째 차원에 접근할 수 있는 .size(n)에 대해서 배워보겠습니다. size를 사용해 출력한 결과는 아래와 같습니다.

In [10]:
print(out.size(0))
print(out.size(1))
print(out.size(2))
print(out.size(3))

1
64
7
7


이제 .view() 함수를 이용하여 tensor를 펼치는 작업을 해보겠습니다.

In [11]:
# 첫번째 차원인 batch size는 그대로 두고 나머지는 펼쳐라.
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


Batch size 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있습니다. 이제 이에 대해 fully-connected layer를 통과시켜보겠습니다. 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환합니다. (10 size의 one hot vector를 이용해 classification 결과를 표현)

In [12]:
fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## 3. CNN으로 MNIST classification
앞서 선언한 도구들을 이용해 모델을 만들고 training한 후 MNIST dataset을 classification 해보도록 하겠습니다.

In [13]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
# 랜덤 시드를 고정합니다. (매번 같은 결과를 얻기 위해서)
torch.manual_seed(777)

In [16]:
# GPU 사용 가능일 경우 랜덤 시드를 고정
if(device == 'cuda'):
    torch.cuda.manual_seed_all(777)

In [17]:
# hyper parameter 고정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [18]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

C:\Users\actgo\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [19]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

이제 모델을 설계합니다.

In [20]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        # first layer
        # ImgIn shape = (?, 28,28, 1)
        # Conv -> (?, 28, 28, 32)
        # Pool -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # second layer
        # ImgIn shape = (?, 14, 14, 32)
        # Conv -> (?, 14, 14, 64)
        # Pool -> (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # fully-connected layer 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층을 위해 flatten
        out = self.fc(out)
        return out

In [21]:
# 모델을 정의합니다.
model = CNN().to(device)

In [22]:
# cost function, optimizer
criterion = nn.CrossEntropyLoss().to(device) # cost function에 soft max 함수 포함되어있음
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
# 총 배치의 수를 출력해보겠습니다.
total_batch = len(data_loader)
print('총 배치의 수 : {}' .format(total_batch))

총 배치의 수 : 600


앞서 batch size를 100으로 설정했는데, 총 배치의 수가 600입니다. 이는 결국 훈련데이터가 총 60,000개 임을 의미합니다.<br>
이제 모델을 training 시켜보겠습니다.

In [24]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader: # 미니 배치 단위로 data를 꺼내옵니다. X는 미니 배치, Y는 label
        # image is already size of (28 x 28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}' .format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225652859
[Epoch:    2] cost = 0.0630322769
[Epoch:    3] cost = 0.0462347269
[Epoch:    4] cost = 0.0374225415
[Epoch:    5] cost = 0.0313504227
[Epoch:    6] cost = 0.0260186009
[Epoch:    7] cost = 0.0217983592
[Epoch:    8] cost = 0.01815827
[Epoch:    9] cost = 0.0158862062
[Epoch:   10] cost = 0.0130476039
[Epoch:   11] cost = 0.0101296436
[Epoch:   12] cost = 0.00962105673
[Epoch:   13] cost = 0.00865442026
[Epoch:   14] cost = 0.00649328902
[Epoch:   15] cost = 0.00665707281


X_test data의 batch size가 10000인데, 이 데이터를 모델에 모두 입력하였더니 runtime error가 발생하였습니다.
이는 아마 허용된 범위를 추가로? 접근을 하려해서 발생한 문제라고 생각하여 test dataset을 100개만 활용하여 accuracy를 확인하도록 코드를 수정했습니다.

In [33]:
# 학습을 진행하지 않기 때문에 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test[:100])
    correct_prediction = torch.argmax(prediction, 1) == Y_test[:100]
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9799999594688416


약 98% 정도의 정확도를 얻을 수 있었습니다.